## Coletando dados da API do site do Pokémon com PySpark

O objetivo desse projeto será coletar os dados da API's disponibilizadas no site oficial do Pokémon para fins acadêmicos seguindo um conceito de criação de dados, trazendo um pouco sobre o que representa aquele dado, a análise exploratória, criação do modelo relacional, criação do Data Lake, processamento dos dados e por fim iremos responder algumas questões.

### Um pouco sobre Pokémon

Pokémon é uma franquia de mídias em geral como jogos, desenhos, filmes que foi criada em 1995. 

A história da franquia é baseada em criaturas de diversas formas e tamanhos conhecidas como Pokémon que podem ser selvagens ou domesticados por humanos, mais conhecidos como "treinadores". No mundo Pokémon as criaturas seriam o equivalente aos animais na vida real. 

Os jogos de Pokémon começaram a ser lançados a partir de 1998 para o Nintendo Game Boy, que foi então, o que impulsionou a popularidade do Pokémon.

Hoje são mais de 8 gerações de jogos lançados oficialmente que são desenvolvidos pela Nintendo e The Pokémon Company.

<p style="text-align:center"><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/001.png" width=300><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/004.png" width=300><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/007.png" width=300></p>

### Uma breve análise exploratória dos dados

Precisamos antes de tudo entender como as API's disponibilizadas no site oficial do jogo de Pokémon. Será consumido as API's https://pokeapi.co/api/v2/pokemon onde encontra-se todos os Pokémon e seus atributos e 
https://pokeapi.co/api/v2/evolution-chain onde encontra-se as evoluções de cada Pokémon.

Então iremos consultar as API's e tentar entender o que é retornado no primeiro request. Para isso iremos importar as bibliotecas requests e json.

In [0]:
import requests
import json

Como podemos observar no script executado abaixo, ele está trazendo um índice de 1154 posições diferentes na key "count", ou seja, ao todo podemos buscar 1154 Pokémon diferentes na base. Como sugerido no retorno da API podemos retornar na mesma consulta todos os 1154 utilizando limit no final do link da API.

In [0]:
requests.get('https://pokeapi.co/api/v2/pokemon').json()

Out[2]: {'count': 1154,
 'next': 'https://pokeapi.co/api/v2/pokemon?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'wee

Se consultarmos o primeiro Pokémon da lista (Bulbasaur) colocando o id no final do link da API, retornaremos diversas keys que são os atributos dos Pokémon que serão utilizadas para criação das tabelas relacionais e para respondermos as perguntas. Na API é retornado as características dos Pokémon, se é apenas de um tipo ou dois tipos (Ex: Pokémon Dragon/Flying), quais golpes eles realizam, suas estatísticas etc.

In [0]:
requests.get('https://pokeapi.co/api/v2/pokemon/1').json()

Out[3]: {'abilities': [{'ability': {'name': 'overgrow',
    'url': 'https://pokeapi.co/api/v2/ability/65/'},
   'is_hidden': False,
   'slot': 1},
  {'ability': {'name': 'chlorophyll',
    'url': 'https://pokeapi.co/api/v2/ability/34/'},
   'is_hidden': True,
   'slot': 3}],
 'base_experience': 64,
 'forms': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}],
 'game_indices': [{'game_index': 153,
   'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}},
  {'game_index': 153,
   'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}},
  {'game_index': 153,
   'version': {'name': 'yellow',
    'url': 'https://pokeapi.co/api/v2/version/3/'}},
  {'game_index': 1,
   'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version/4/'}},
  {'game_index': 1,
   'version': {'name': 'silver',
    'url': 'https://pokeapi.co/api/v2/version/5/'}},
  {'game_index': 1,
   'version': {'name': 'crystal',
    'url': 'https://pok

Mesma situação acontece se consultarmos a API de evoluções, porém na key "count" são retornado 468 Pokémon que evoluem 1 ou mais vezes.

In [0]:
requests.get('https://pokeapi.co/api/v2/evolution-chain').json()

Out[4]: {'count': 468,
 'next': 'https://pokeapi.co/api/v2/evolution-chain?offset=20&limit=20',
 'previous': None,
 'results': [{'url': 'https://pokeapi.co/api/v2/evolution-chain/1/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/2/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/3/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/4/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/5/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/6/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/7/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/8/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/9/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/10/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/11/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/12/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/13/'},
  {'url': 'https://pokeapi.co/api/v2/evolution-chain/14/'},
  {'url': 'https://pokeapi.co/api/v2/evolut

Como podemos observar abaixo no final da consulta da API, ela retorna no final do request a pré-evolução, seguida da primeira evolução e depois da terceira. Os Pokémon por padrão conseguem evoluir apenas duas vezes, sendo que em alguns casos como os Pokémon Eevee e Poliwag podem possuir mais de um caminho evolutivo. Essa informação nos ajudará quando formos criar o dataset de evolução.

In [0]:
requests.get('https://pokeapi.co/api/v2/evolution-chain/1').json()

Out[5]: {'baby_trigger_item': None,
 'chain': {'evolution_details': [],
  'evolves_to': [{'evolution_details': [{'gender': None,
      'held_item': None,
      'item': None,
      'known_move': None,
      'known_move_type': None,
      'location': None,
      'min_affection': None,
      'min_beauty': None,
      'min_happiness': None,
      'min_level': 16,
      'needs_overworld_rain': False,
      'party_species': None,
      'party_type': None,
      'relative_physical_stats': None,
      'time_of_day': '',
      'trade_species': None,
      'trigger': {'name': 'level-up',
       'url': 'https://pokeapi.co/api/v2/evolution-trigger/1/'},
      'turn_upside_down': False}],
    'evolves_to': [{'evolution_details': [{'gender': None,
        'held_item': None,
        'item': None,
        'known_move': None,
        'known_move_type': None,
        'location': None,
        'min_affection': None,
        'min_beauty': None,
        'min_happiness': None,
        'min_level': 32,
     

Após entendermos como funciona a API iremos criar uma lista que retorna os links dos Pokémon para utilizarmos na criação do Data Lake

In [0]:
#Criando a lista de links dos Pokemons
pokemon_list = []
pokemon = json.loads(requests.get('https://pokeapi.co/api/v2/pokemon?limit=1154').content)
for i in pokemon['results']:
    pokemon_list.append([i["name"],i["url"]])

In [0]:
print("Foram retornados", len(pokemon_list), "Pokémon")

Foram retornados 1154 Pokémon


In [0]:
#Criando a lista de links dos Pokemons que evoluem
evolution_list = []
evolution = json.loads(requests.get('https://pokeapi.co/api/v2/evolution-chain?limit=468').content)
for i in evolution['results']:
    evolution_list.append(i["url"])

In [0]:
print("Foram retornados", len(evolution_list), "Pokémon que evoluem")

Foram retornados 468 Pokémon que evoluem


### Criando o modelo relacional

Com a análise exploratória realizada em cima dos arquivos json, permitiu que realizassemos a criação do modelo relacional. 

O modelo como a imagem mostra abaixo estará separado por 5 datasets, sendo o pokemon a tabela de fato e as demais tabelas de dimensões.

<p style="text-align:center"><img src="https://drive.google.com/uc?export=view&id=17Z_KKR_Sn9c-xlBz77csPXC60NLwXjOg"></p>

### Criando o Data Lake

Com todos os links das listas de Pokémon e suas evoluções já identificados e realizado a criação do modelo relacional, iremos agora realizar a criação do Data Lake consultando a API e armazenando os dados retornados na FileStore do Databricks.

A criação de um Data Lake permite que outras informações que podem ser coletadas futuramente nas API's como habilidades, se foi necessário utilizar algum item na evolução etc.

Primeiramente iremos armazenar os dados retornados da primeira API, aonde é retornado todas as informações dos Pokémon menos sua evolução.

In [0]:
for index, i in enumerate(pokemon_list):
    pokemon_dump = json.dumps(requests.get(pokemon_list[index][1]).json())
    dbutils.fs.put('/FileStore/datalake/pokemons/'+ pokemon_list[index][0] +'.json', pokemon_dump, True)

Wrote 231951 bytes.
Wrote 203158 bytes.
Wrote 238859 bytes.
Wrote 254702 bytes.
Wrote 223160 bytes.
Wrote 292596 bytes.
Wrote 259257 bytes.
Wrote 227923 bytes.
Wrote 272590 bytes.
Wrote 29780 bytes.
Wrote 27657 bytes.
Wrote 222359 bytes.
Wrote 27754 bytes.
Wrote 26639 bytes.
Wrote 196676 bytes.
Wrote 182985 bytes.
Wrote 169029 bytes.
Wrote 181566 bytes.
Wrote 230654 bytes.
Wrote 231192 bytes.
Wrote 186718 bytes.
Wrote 180192 bytes.
Wrote 220944 bytes.
Wrote 220430 bytes.
Wrote 229451 bytes.
Wrote 202970 bytes.
Wrote 253577 bytes.
Wrote 248253 bytes.
Wrote 233518 bytes.
Wrote 205593 bytes.
Wrote 295069 bytes.
Wrote 234196 bytes.
Wrote 204433 bytes.
Wrote 293654 bytes.
Wrote 325665 bytes.
Wrote 287060 bytes.
Wrote 234270 bytes.
Wrote 182722 bytes.
Wrote 292551 bytes.
Wrote 269314 bytes.
Wrote 228469 bytes.
Wrote 216176 bytes.
Wrote 197565 bytes.
Wrote 187651 bytes.
Wrote 169821 bytes.
Wrote 227858 bytes.
Wrote 211107 bytes.
Wrote 189504 bytes.
Wrote 211135 bytes.
Wrote 214533 bytes.
Wrot

Agora iremos armazenar os dados retornados da segunda API, onde é retornado apenas a evolução dos Pokémon.

In [0]:
for index, i in enumerate(evolution_list):
    evolution_json = requests.get(evolution_list[index]).json()
    evolution_dump = json.dumps(evolution_json)
    dbutils.fs.put('/FileStore/datalake/evolutions/evolution_'+ str(evolution_json['chain']['species']['name']) +'.json', evolution_dump, True)

Wrote 1416 bytes.
Wrote 1421 bytes.
Wrote 1418 bytes.
Wrote 1419 bytes.
Wrote 1414 bytes.
Wrote 1417 bytes.
Wrote 1275 bytes.
Wrote 805 bytes.
Wrote 802 bytes.
Wrote 1483 bytes.
Wrote 1339 bytes.
Wrote 1487 bytes.
Wrote 1486 bytes.
Wrote 1484 bytes.
Wrote 1401 bytes.
Wrote 1491 bytes.
Wrote 1415 bytes.
Wrote 2160 bytes.
Wrote 806 bytes.
Wrote 808 bytes.
Wrote 807 bytes.
Wrote 1886 bytes.
Wrote 807 bytes.
Wrote 807 bytes.
Wrote 875 bytes.
Wrote 2164 bytes.
Wrote 1414 bytes.
Wrote 1415 bytes.
Wrote 1491 bytes.
Wrote 812 bytes.
Wrote 1415 bytes.
Wrote 807 bytes.
Wrote 2553 bytes.
Wrote 3564 bytes.
Wrote 825 bytes.
Wrote 804 bytes.
Wrote 804 bytes.
Wrote 802 bytes.
Wrote 875 bytes.
Wrote 1414 bytes.
Wrote 868 bytes.
Wrote 805 bytes.
Wrote 806 bytes.
Wrote 811 bytes.
Wrote 878 bytes.
Wrote 1276 bytes.
Wrote 2033 bytes.
Wrote 877 bytes.
Wrote 809 bytes.
Wrote 1483 bytes.
Wrote 1555 bytes.
Wrote 880 bytes.
Wrote 201 bytes.
Wrote 1483 bytes.
Wrote 809 bytes.
Wrote 874 bytes.
Wrote 1545 bytes.


### Processamento de dados

Após termos criado o modelo relacional e o Data Lake, agora iremos processar os arquivos .json com o PySpark que nos permite processar grandes volumes de dados, interpreta-los e transforma-los em datasets.

Para isso primeiramente iremos importar a biblioteca pyspark.sql.functions que servirá para executar algumas funções específicas como selecionar uma coluna e alterar o nome dela, etc.

In [0]:
import pyspark.sql.functions as F

Com a biblioteca importada iremos começar a processar todos os arquivos .json retornados na primeira API para que a gente possa posteriormente realizar a criação dos dataset pokemon, types, moves e stats.

In [0]:
rawDF = spark.read.json("dbfs:/FileStore/datalake/pokemons/*.json", multiLine = "true")

Após o término do processamento iremos realizar a criação dos dataset retornados na API de acordo com o modelo relacional proposto anteriormente.

In [0]:
pokemon = rawDF.select("id", "name", "base_experience", "height", "weight", "is_default")
pokemon = pokemon.toPandas()

pokemon.head()

,id,name,base_experience,height,weight,is_default
0,151,mew,300.0,4,40,True
1,150,mewtwo,340.0,20,1220,True
2,113,chansey,395.0,11,346,True
3,122,mr-mime,161.0,13,545,True
4,149,dragonite,300.0,22,2100,True


In [0]:
types = rawDF.select("id", F.col("types.type.name")[0].alias("first_type"), F.col("types.type.name")[1].alias("second_type"))
types = types.toPandas()

types.head()

,id,first_type,second_type
0,151,psychic,None
1,150,psychic,None
2,113,normal,None
3,122,psychic,fairy
4,149,dragon,flying


In [0]:
moves = rawDF.withColumn("new", F.arrays_zip("moves"))\
       .withColumn("new", F.explode("new.moves"))\
       .withColumn("newversion", F.explode("new.version_group_details"))\
       .select("id", F.col("new.move.name").alias("move_name"),\
              F.col("newversion.move_learn_method.name").alias("move_learn_method_name"),\
              F.col("newversion.version_group.name").alias("version_group_name"))
moves = moves.toPandas()

moves.head()

,id,move_name,move_learn_method_name,version_group_name
0,151,pound,level-up,red-blue
1,151,pound,level-up,yellow
2,151,pound,level-up,gold-silver
3,151,pound,level-up,crystal
4,151,pound,level-up,ruby-sapphire


In [0]:
stats = rawDF.withColumn("new", F.arrays_zip("stats"))\
       .withColumn("new", F.explode("new.stats"))\
       .select("id", F.col("new.base_stat").alias("base_stat"),  F.col("new.stat.name").alias("stat_name"))
stats = stats.toPandas()

stats.head()

,id,base_stat,stat_name
0,151,100,hp
1,151,100,attack
2,151,100,defense
3,151,100,special-attack
4,151,100,special-defense


Finalizado a criação dos dataset que foram baseados na primeira API, iremos realizar o processamento dos dados retornados referente a segunda API.

In [0]:
rawDFB = spark.read.json("dbfs:/FileStore/datalake/evolutions/*.json", multiLine = "true")

Com o processo terminado iremos realizar a criação do nosso último dataset, o dataset de evolução. 

Para esse caso em específico devido as complexidades da estrutura do arquivo JSON iremos separar em 4 etapas, sendo uma delas baseado no Pokémon base para sua primeira evolução, a outra baseado na primeira evolução para segunda evolução, a junção das duas primeiras etapas, retornar o id da primeira API para depois inclui-lo no dataset pois não há a key id na segunda API correspondente a cada Pokémon e aí finalizaremos com a criação do dataset.

##### Primeira etapa:

In [0]:
evolutionA = rawDFB.withColumn("new", F.arrays_zip("chain.evolves_to"))\
.withColumn("new", F.explode("new.evolves_to"))\
.select(F.col("chain.species.name").alias("name"), F.col("new.species.name").alias("evol"))

In [0]:
evolutionB = rawDFB.withColumn("new", F.arrays_zip("chain.evolves_to"))\
.withColumn("new", F.explode("new.evolves_to"))\
.withColumn("newb", F.explode("new.evolves_to"))\
.select(F.col("new.species.name").alias("name"), F.col("newb.species.name").alias("evol"))

##### Segunda etapa:

In [0]:
evolution = evolutionA.union(evolutionB)

##### Terceira etapa:

In [0]:
poke_id = rawDF.select("id", F.col("name").alias("name_temp"))

##### Quarta etapa:

In [0]:
evolution = evolution.join(poke_id,poke_id.name_temp == evolution.name,"inner")
evolution = evolution.select(F.col("id"),F.col("name"), "evol")
evolution = evolution.toPandas()

evolution.head()

,id,name,evol
0,113,chansey,blissey
1,122,mr-mime,mr-rime
2,35,clefairy,clefable
3,112,rhydon,rhyperior
4,108,lickitung,lickilicky


### Respondendo perguntas sobre os dados

Com os datasets criados, podemos responder algumas perguntas. Para isso, iremos importar a biblioteca pandas que nos auxiliará a encontrar a resposta correta. Vamos lá!

In [0]:
import pandas as pd

##### Quais são os maiores Pokémon? Exiba os Pokémon e a altura.

In [0]:
max_height = pokemon.copy()
max_height = max_height[["name", "height"]]\
.loc[max_height['height'] == max_height['height'].max()]\
.rename(columns={'name': 'Pokémon', 'height': 'Altura'})
max_height

,Pokémon,Altura
1045,eternatus-eternamax,1000


##### Quantos Pokémon possuem mais de um caminho evolutivo, ou seja, mais de uma segunda ou terceira forma? Informe a quantidade, tomando como base o número distinto de Pokémon na sua primeira forma.

In [0]:
resultEvol = evolution.copy()
resultEvol['count'] = resultEvol.groupby('name')['evol'].transform('count')
resultEvol = resultEvol.loc[resultEvol['count'] > 1]
resultEvol = resultEvol['name'].drop_duplicates()
print("O número de Pokémon com mais de um caminho evolutivo é", resultEvol.count())

O número de Pokémon com mais de um caminho evolutivo é 15


##### Quais são os Pokémon do tipo gelo (ice) que mais fornecem experiência ao serem derrotados? Levando em consideração apenas os Pokémon que estão em sua forma padrão, exibindo os Pokémon e a experiência que é fornecida.

In [0]:
resultExp = pd.merge(pokemon,types,on='id')
resultExp = resultExp.loc[(resultExp['first_type'] == 'ice') & (resultExp['is_default'] == True) & (resultExp['second_type'].isna())]
resultExp = resultExp.loc[resultExp['base_experience'] == resultExp['base_experience'].max()]
resultExp = resultExp[["name", "base_experience"]].rename(columns={'name': 'Pokémon', 'base_experience': 'Experiência'})
resultExp

,Pokémon,Experiência
271,regice,290.0
1021,glastrier,290.0


##### Encontre o golpe (move) que é mais aprendido por Pokémon pelo método level-up e em qual versão do jogo isso acontece, levando em consideração apenas os Pokémon que estão em sua forma padrão. Em seguida, entre os Pokémon que aprendem esse golpe (por level-up e na versão do jogo encontrada no passo anterior), diga quais são os Pokémon que possuem o maior valor do atributo (stat) ‘attack’. Exiba os Pokémon, a versão do jogo em que acontece de mais Pokémon aprenderem o golpe e qual é o valor do atributo ‘attack’ deles.

In [0]:
resultMove = moves.copy()
resultMove = resultMove.loc[resultMove['move_learn_method_name'] == 'level-up']
resultMove['count'] = resultMove.groupby(['move_name', 'version_group_name'])['move_name'].transform('count')
resultMove = resultMove.loc[resultMove['count'] == resultMove['count'].max()]
resultMoveMerge = pd.merge(pokemon,resultMove,on='id')
resultMoveMerge = resultMoveMerge.loc[(resultMoveMerge['is_default'] == True)]
final_resultMove = pd.merge(resultMoveMerge,stats,on='id')
final_resultMove = final_resultMove.loc[final_resultMove['stat_name'] == 'attack']
final_resultMove = final_resultMove.loc[final_resultMove['base_stat'] == final_resultMove['base_stat'].max()]
final_resultMove = final_resultMove[['name','version_group_name','base_stat']].rename(columns={'name': 'Pokémon', 'version_group_name': 'Versão', 'base_stat': 'Valor do Atributo'})
final_resultMove
final_resultMove

,Pokémon,Versão,Valor do Atributo
733,darmanitan-standard,ultra-sun-ultra-moon,140


##### Quais são as 7 evoluções que geram o maior aumento em algum dos atributos? Por exemplo, um Pokémon com o stat ‘attack’ igual a 50 evoluir para um com o stat ‘attack’ igual a 150, o aumento seria de 100. Somente aqueles Pokémon na sua forma padrão. Exiba a pré-evolução, a evolução, o atributo afetado e o quanto ele aumentou com a evolução.

In [0]:
resultEvolStat = pd.merge(pokemon, stats,on='id')
resultEvolStat = resultEvolStat.loc[resultEvolStat['is_default'] == True]
ta = resultEvolStat.copy()
ta = pd.merge(resultEvolStat, evolution,on='id')
tb = pd.merge(ta, resultEvolStat, left_on='evol', right_on='name')
tb = tb.loc[tb['stat_name_x'] == tb['stat_name_y']]
tb['diff'] = tb['base_stat_y'] - tb['base_stat_x']
tb = tb.sort_values('diff', ascending=False)
tb = tb[['name_x','evol','stat_name_x','diff']].rename(columns={'name_x': 'Pré-evolução', 'evol': 'Evolução', 'stat_name_x': 'Atributo', 'diff': 'Aumento do atributo'})

tb.drop_duplicates('Atributo')

,Pré-evolução,Evolução,Atributo,Aumento do atributo
7704,happiny,chansey,hp,150
10799,shelmet,accelgor,speed,120
13111,magikarp,gyarados,attack,115
15069,cosmoem,lunala,special-attack,108
13334,wimpod,golisopod,defense,100
15148,cosmog,cosmoem,special-defense,100


##### Quais são os Pokémon com maior vantagem atacante sobre outros Pokémon? Leve em consideração apenas o tipo do Pokémon (Um Pokémon do tipo fogo/lutador (fire/fighting) possui vantagem atacante em um Pokémon inseto/voador (bug/flying), pois o tipo fogo tem vantagem sobre inseto, apesar do tipo lutador não ser eficiente contra o tipo inseto e voador) e apenas os Pokémon que estão em sua forma padrão. Exiba os Pokémon, e a quantidade de Pokémon que eles possuem vantagem atacante sobre.

In [0]:
resultPokeAttack = pd.merge(pokemon,stats,on='id')
resultPokeAttack = pd.merge(resultPokeAttack,types,on='id')
resultPokeAttack = resultPokeAttack.loc[(resultPokeAttack['is_default'] == True) & (resultPokeAttack['stat_name'] == 'attack')]
resultPokeAttack_ta = resultPokeAttack.copy()
resultPokeAttack_ta = resultPokeAttack_ta.loc[(resultPokeAttack_ta['first_type'] == 'fire') & (resultPokeAttack_ta['second_type'] == 'fighting')]
resultPokeAttack_ta = resultPokeAttack_ta[['id','name','base_stat']]

resultPokeAttack_tb = resultPokeAttack.copy()
resultPokeAttack_tb = resultPokeAttack_tb.loc[(resultPokeAttack_tb['first_type'] == 'bug') & (resultPokeAttack_tb['second_type'] == 'flying')]
resultPokeAttack_tb = resultPokeAttack_tb [['id','name','base_stat']]

final_ResultPokeAttack = pd.merge(resultPokeAttack_ta, resultPokeAttack_tb, how='cross')
final_ResultPokeAttack = final_ResultPokeAttack.loc[final_ResultPokeAttack['base_stat_x'] > final_ResultPokeAttack['base_stat_y']]
final_ResultPokeAttack['count'] = final_ResultPokeAttack.groupby('name_x')['name_y'].transform('count')
final_ResultPokeAttack = final_ResultPokeAttack.loc[final_ResultPokeAttack['count'] == final_ResultPokeAttack['count'].max()]
final_ResultPokeAttack = final_ResultPokeAttack[['name_x', 'count']].rename(columns={'name_x': 'Pokémon', 'count': 'Soma'})
final_ResultPokeAttack.drop_duplicates()

,Pokémon,Soma
0,blaziken,13
52,emboar,13


##### Quais são os maiores Pokémon considerando apenas os que estão em sua forma padrão?

In [0]:
max_heightB = pokemon.copy()
max_heightB = max_heightB.loc[max_heightB['is_default'] == True]
max_heightB = max_heightB[["name", "height"]]\
.loc[max_heightB['height'] == max_heightB['height'].max()]\
.rename(columns={'name': 'Pokémon', 'height': 'Altura'})
max_heightB

,Pokémon,Altura
1033,eternatus,200


##### Quais são os Pokémon mais pesados considerando apenas os que estão em sua forma padrão?

In [0]:
max_weight = pokemon.copy()
max_weight = max_weight.loc[max_weight['is_default'] == True]
max_weight = max_weight[["name", "weight"]]\
.loc[max_weight['weight'] == max_weight['weight'].max()]\
.rename(columns={'name': 'Pokémon', 'weight': 'Peso'})
max_weight

,Pokémon,Peso
837,celesteela,9999
1089,cosmoem,9999


### Referências

* https://www.pokemon.com/br/guia-para-pais/
* https://pokeapi.co/docs/v2
* https://spark.apache.org/docs/2.3.1/api/python/index.html
* https://pandas.pydata.org/docs/user_guide/index.html
* https://medium.com/expedia-group-tech/working-with-json-in-apache-spark-1ecf553c2a8c